In [18]:
import tensorflow as tf
from tensorflow import keras

In [31]:
def qbert_model(vocab_size, num_layers, dff, d_model, num_heads, dropout, name = 'qbert'):

    input = tf.keras.layers.Input(shape = (None, ), name = 'inputs')

    padding_mask = tf.keras.Input(shape = (1, 1, None), name = 'padding_mask')

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(input)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    print(embeddings.shape)
    pos_layer = tf.random.normal(shape = (None, None, d_model))
    embeddings += tf.kears.layers.add(embeddings, pos_layer)
    outputs = tf.keras.layers.Dropout(rate = dropout)(embeddings)


#     for i in range(num_layers) :

#         outputs = encoder_layer(dff = dff
#                                 , d_model=d_model
#                                 , num_heads = num_heads
#                                 , dropout = dropout
#                                 , name = 'encoding_layer_{}'.format(i))([outputs, padding_mask])

    return tf.keras.Model(inputs = [input, padding_mask], outputs = outputs, name = name)


In [32]:
qbert_model(32000, 1, 1, 200, 1, 0.1)

(None, None, 200)


ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.